In [1]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBRegressor
import numpy as np
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
# Database Connection
server = 'EFGSVR050\\NOVY'
database = 'Olist Case Study'
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database}')

# Query to fetch data
query = """
SELECT 
    p.product_category_name, 
    YEAR(o.order_purchase_timestamp) AS order_year, 
    MONTH(o.order_purchase_timestamp) AS order_month, 
    SUM(op.payment_value) AS total_payment_value,
    COUNT(o.order_id) AS total_orders
FROM 
    olist_orders_dataset AS o
INNER JOIN 
    olist_order_payments_dataset AS op
    ON o.order_id = op.order_id
INNER JOIN 
    olist_order_items_dataset AS oi
    ON o.order_id = oi.order_id
INNER JOIN 
    olist_products_dataset AS p
    ON oi.product_id = p.product_id
WHERE 
    o.order_status = 'delivered'
GROUP BY 
    p.product_category_name, 
    YEAR(o.order_purchase_timestamp), 
    MONTH(o.order_purchase_timestamp)
ORDER BY 
    order_year, order_month;

"""

# Fetching data into DataFrame
data = pd.read_sql(query, conn)
conn.close()

In [3]:
# Combine order_year and order_month into a single column called SalesMonth
data['SalesMonth'] = data['order_year'].astype(str) + '-' + data['order_month'].astype(str).str.zfill(2)

In [4]:
# Convert SalesMonth to datetime
data['SalesMonth'] = pd.to_datetime(data['SalesMonth'], format='%Y-%m')

# Sort data by SalesMonth
data = data.sort_values(by='SalesMonth')

# Encode product categories
label_encoder = LabelEncoder()
data['product_category_encoded'] = label_encoder.fit_transform(data['product_category_name'])

In [5]:
data.head()

,product_category_name,order_year,order_month,total_payment_value,total_orders,SalesMonth,product_category_encoded
0,fashion_calcados,2016,10,40.95,1,2016-10-01,34
29,fashion_bolsas_e_acessorios,2016,10,585.24,8,2016-10-01,33
28,climatizacao,2016,10,3872.83,8,2016-10-01,19
27,pet_shop,2016,10,775.86,5,2016-10-01,63
26,cama_mesa_banho,2016,10,2291.71,8,2016-10-01,13


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1272 entries, 0 to 1271
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   product_category_name     1251 non-null   object        
 1   order_year                1272 non-null   int64         
 2   order_month               1272 non-null   int64         
 3   total_payment_value       1272 non-null   float64       
 4   total_orders              1272 non-null   int64         
 5   SalesMonth                1272 non-null   datetime64[ns]
 6   product_category_encoded  1272 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(1)
memory usage: 74.5+ KB


In [7]:
# Drop NaN values
data = data.dropna()

In [8]:
# Create lag features for forecasting
data['PrevMonthSales'] = data.groupby('product_category_encoded')['total_payment_value'].shift(1)
data['Prev2MonthSales'] = data.groupby('product_category_encoded')['total_payment_value'].shift(2)
data['PrevQuarterSales'] = data.groupby('product_category_encoded')['total_payment_value'].shift(3)
data['PrevYearSales'] = data.groupby('product_category_encoded')['total_payment_value'].shift(12)

In [9]:
data.head()

,product_category_name,order_year,order_month,total_payment_value,total_orders,SalesMonth,product_category_encoded,PrevMonthSales,Prev2MonthSales,PrevQuarterSales,PrevYearSales
0,fashion_calcados,2016,10,40.95,1,2016-10-01,34,NaN,NaN,NaN,NaN
29,fashion_bolsas_e_acessorios,2016,10,585.24,8,2016-10-01,33,NaN,NaN,NaN,NaN
28,climatizacao,2016,10,3872.83,8,2016-10-01,19,NaN,NaN,NaN,NaN
27,pet_shop,2016,10,775.86,5,2016-10-01,63,NaN,NaN,NaN,NaN
26,cama_mesa_banho,2016,10,2291.71,8,2016-10-01,13,NaN,NaN,NaN,NaN


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1251 entries, 0 to 1271
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   product_category_name     1251 non-null   object        
 1   order_year                1251 non-null   int64         
 2   order_month               1251 non-null   int64         
 3   total_payment_value       1251 non-null   float64       
 4   total_orders              1251 non-null   int64         
 5   SalesMonth                1251 non-null   datetime64[ns]
 6   product_category_encoded  1251 non-null   int32         
 7   PrevMonthSales            1178 non-null   float64       
 8   Prev2MonthSales           1105 non-null   float64       
 9   PrevQuarterSales          1033 non-null   float64       
 10  PrevYearSales             428 non-null    float64       
dtypes: datetime64[ns](1), float64(5), int32(1), int64(3), object(1)
memory usage: 112.4+ KB

In [13]:
train_data = data.dropna(subset=['product_category_encoded', 'PrevMonthSales', 'Prev2MonthSales', 'PrevQuarterSales', 'PrevYearSales','total_payment_value'])
X_train = train_data[['product_category_encoded', 'PrevMonthSales', 'Prev2MonthSales', 'PrevQuarterSales', 'PrevYearSales']]
y_train = train_data['total_payment_value']

In [14]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

In [15]:
# Train the XGBoost model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [16]:
# Example structure for future_template
future_template = pd.DataFrame(X_train)
# Predicting revenue for 2025
future_template['Predicted_Sales'] = xgb_model.predict(
    future_template[['product_category_encoded', 'PrevMonthSales', 'Prev2MonthSales', 'PrevQuarterSales', 'PrevYearSales']]
)

In [17]:
# Ensure lag features are calculated
data['PrevMonthSales'] = data.groupby('product_category_encoded')['total_payment_value'].shift(1)
data['Prev2MonthSales'] = data.groupby('product_category_encoded')['total_payment_value'].shift(2)
data['PrevQuarterSales'] = data.groupby('product_category_encoded')['total_payment_value'].shift(3)
data['PrevYearSales'] = data.groupby('product_category_encoded')['total_payment_value'].shift(12)

# Get the latest data for each category
recent_data = data.groupby('product_category_encoded').tail(1).copy()

# Define the number of months to predict
num_months = 6

# Initialize placeholders for predictions
future_predictions = []

# Iteratively generate predictions
for month in range(1, num_months + 1):
    # Prepare input features for all products in parallel
    recent_data['PredictedSales'] = xgb_model.predict(
        scaler.transform(
            recent_data[['product_category_encoded', 'PrevMonthSales', 'Prev2MonthSales', 
                         'PrevQuarterSales', 'PrevYearSales']].fillna(0)  # Handle missing lagged features
        )
    )

    # Store predictions
    recent_data['Month'] = f'Month {month}'
    future_predictions.append(recent_data[['product_category_encoded', 'PredictedSales', 'Month']].copy())

    # Update lagged features for the next iteration
    recent_data['PrevYearSales'] = recent_data['PrevYearSales'].shift(1, fill_value=0)
    recent_data['PrevQuarterSales'] = recent_data['PrevQuarterSales'].shift(1, fill_value=0)
    recent_data['Prev2MonthSales'] = recent_data['PrevMonthSales']
    recent_data['PrevMonthSales'] = recent_data['PredictedSales']

# Combine all predictions
future_df = pd.concat(future_predictions, ignore_index=True)

# Map product category names back to the encoded IDs
future_df['product_category_name'] = future_df['product_category_encoded'].map(
    dict(zip(data['product_category_encoded'], data['product_category_name']))
)

# Display final predictions
print(future_df)

# Save predictions for future use
future_df.to_csv('predicted_sales_2025.csv', index=False)


     product_category_encoded  PredictedSales    Month  \
0                          67     1865.901978  Month 1   
1                          46      974.709656  Month 1   
2                          17      648.732971  Month 1   
3                          69     -360.689087  Month 1   
4                          37      985.218140  Month 1   
..                        ...             ...      ...   
433                         1    19363.421875  Month 6   
434                        64    30262.718750  Month 6   
435                        34    21158.109375  Month 6   
436                         7     4974.499023  Month 6   
437                        65    16172.510742  Month 6   

                             product_category_name  
0                               seguros_e_servicos  
1                                       la_cuisine  
2                                cds_dvds_musicais  
3                         tablets_impressao_imagem  
4                    fashion_roupa_inf